# Calibrate cameras, create initial orthoimage and partial DSM

In [ ]:
import os
from glob import glob
import subprocess
import numpy as np
from tqdm import tqdm
import pandas as pd
import cv2
from shapely.geometry import Point
import geopandas as gpd
import shutil
# Ignore warnings (rasterio throws a warning whenever an image is not georeferenced. Annoying in this case.)
import warnings
warnings.filterwarnings('ignore')

# Locate image files
data_folder = '/Users/rdcrlrka/Research/Soo_locks'
image_folder = os.path.join(data_folder, '20251001_imagery', 'frames_IR')
image_list = sorted(glob(os.path.join(image_folder, '*.tiff')))
print(f"{len(image_list)} images located")

# Grab other input files
refdem_file = os.path.join(os.getcwd(), '..', 'inputs', '20251001_Soo_Model_1cm_mean_UTM19N-fake_filled_cropped.tif')
gcp_folder = os.path.join(os.getcwd(), '..', 'inputs', 'gcp')
cams_file = os.path.join(os.getcwd(), '..', 'inputs', 'cams_lonlat-fake.txt')

# Define output folders
out_folder = image_folder + '_proc_out'
os.makedirs(out_folder, exist_ok=True)
new_image_folder = os.path.join(out_folder, 'single_band_images')
undistorted_folder = os.path.join(out_folder, 'undistorted_images_cams')
init_ortho_folder = os.path.join(out_folder, 'init_ortho')
init_stereo_folder = os.path.join(out_folder, 'init_stereo')
ba_folder = os.path.join(out_folder, 'bundle_adjust')
final_ortho_folder = os.path.join(out_folder, 'final_ortho')
final_stereo_folder = os.path.join(out_folder, 'final_stereo')

## Merge GCP

In [ ]:
gcp_merged_file = os.path.join(gcp_folder, 'GCP_merged.csv')
if not os.path.exists(gcp_merged_file):

    gcp_list = sorted(glob(os.path.join(gcp_folder, '*.gcp')))
    df_list = []
    for gcp_file in gcp_list:
        df = pd.read_csv(
            gcp_file,
            sep=',',
            header=None,
            skiprows=[0],
            names=[
                'point_index', 'lat', 'lon', 'Z', 'lat_sigma', 'lon_sigma', 'Z_sigma', 
                'image_path', 'col_sample', 'row_sample', 'use_lat', 'use_lon']
        )
        df_list += [df]

    dfs = pd.concat(df_list).reset_index(drop=True)

    # reproject to UTM zone 19N
    gdf = gpd.GeoDataFrame(
        dfs,
        geometry=[Point(x,y) for x,y in dfs[['lon', 'lat']].values],
        crs="EPSG:4326"
    )
    gdf = gdf.to_crs("EPSG:32619")
    gdf['X'] = [x.coords.xy[0][0] for x in gdf['geometry']]
    gdf['Y'] = [x.coords.xy[1][0] for x in gdf['geometry']]

    # use just the image file name
    gdf['image_name'] = [os.path.basename(x) for x in gdf['image_path']]

    # select relevant columns
    gdf = gdf[['image_name', 'X', 'Y', 'Z', 'col_sample', 'row_sample']]

    # save to file
    gdf.to_csv(gcp_merged_file, sep=',', index=False)
    print('Saved merged GCP:', gcp_merged_file)
else:
    print('Merged GCP already exists in file, skipping merge.')


# Reproject from UTM zone 19 N to ECEF for use in ASP
gcp_merged_ecef_file = os.path.join(gcp_folder, 'GCP_merged_ECEF.csv')
if not os.path.exists(gcp_merged_ecef_file):
    # Load the merged file
    gcp_merged = pd.read_csv(gcp_merged_file, sep=',')

    # Reproject from UTM to ECEF
    geom = [Point(x,y,z) for x,y,z in gcp_merged[['X', 'Y', 'Z']].values]
    gdf = gpd.GeoDataFrame(geometry=geom, crs="EPSG:32619")
    gdf = gdf.to_crs("EPSG:4978")
    gcp_merged['X'] = [x.x for x in gdf['geometry']]
    gcp_merged['Y'] = [x.y for x in gdf['geometry']]
    gcp_merged['Z'] = [x.z for x in gdf['geometry']]

    # Save to file
    gcp_merged.to_csv(gcp_merged_ecef_file, sep=',', index=False)
    print('Saved merged GCP in ECEF coordinates:', gcp_merged_ecef_file)
else:
    print('Merged GCP in ECEF coordinates already exists in file, skipping reprojection.')


## Convert images to single band in case they're RGB

A couple IR images (near the windows) were captured in RGB 

In [ ]:
os.makedirs(new_image_folder, exist_ok=True)

# iterate over images
print('Saving single-band images to:', new_image_folder)
for image_file in tqdm(image_list):
    # convert images to single band
    out_fn = os.path.join(new_image_folder, os.path.basename(image_file))
    if os.path.exists(out_fn):
        continue
    cmd = [
        "gdal_translate",
        "-b", "1",
        image_file, out_fn
    ]
    subprocess.run(cmd)

## Calibrate cameras using GCP

In [ ]:
def save_tsai(tsai_dict, filename):
    """
    Save a TSAI (.tsai) pinhole camera file from a dictionary.
    """
    with open(filename, 'w') as f:
        f.write("VERSION_4\n")
        f.write("PINHOLE\n")
        f.write(f"fu = {tsai_dict['fu']}\n")
        f.write(f"fv = {tsai_dict['fv']}\n")
        f.write(f"cu = {tsai_dict['cu']}\n")
        f.write(f"cv = {tsai_dict['cv']}\n")
        f.write(f"u_direction = {' '.join(map(str, tsai_dict['u_direction']))}\n")
        f.write(f"v_direction = {' '.join(map(str, tsai_dict['v_direction']))}\n")
        f.write(f"w_direction = {' '.join(map(str, tsai_dict['w_direction']))}\n")
        f.write(f"C = {' '.join(map(str, tsai_dict['C']))}\n")
        f.write("R = " + " ".join(map(str, tsai_dict['R'].flatten())) + "\n")
        f.write(f"pitch = {tsai_dict['pitch']}\n")
        # Add small distortion for bundle adjust
        f.write("TSAI\n")
        f.write("k1 = -1e-6\nk2 = 1e-6\np1 = 0\np2 = 0\nk3 = 1e-6\n")


def calibrate_shared_intrinsics(image_files, gcp_file, output_folder=None, file_prefix=None):
    object_points_list = []
    image_points_list = []
    image_size = None

    # --- Load merged GCP ---
    gcp = pd.read_csv(gcp_file, sep=',')    

    # --- Compile GCP (object) and image (pixel) points --- 
    for image_file in image_files:
        # Subset GCP to image
        gcp_image = gcp.loc[gcp['image_name']==os.path.basename(image_file)]

        # Object and image points
        obj_pts = gcp_image[['X','Y','Z']].values.astype(np.float32)
        image_pts = gcp_image[['col_sample','row_sample']].values.astype(np.float32)
        obj_pts = obj_pts.reshape(-1,1,3)
        image_pts = image_pts.reshape(-1,1,2)

        object_points_list.append(obj_pts)
        image_points_list.append(image_pts)

        # get image size
        if image_size is None:
            image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
            image_size = (image.shape[1], image.shape[0])

    if len(object_points_list) == 0:
        raise ValueError("No valid images for calibration")
    
    # --- Subtract the mean of all object points for better calculation ---
    all_obj_pts = np.vstack([op.reshape(-1,3) for op in object_points_list])
    object_points_mean = all_obj_pts.mean(axis=0)
    object_points_list = [x - object_points_mean for x in object_points_list]

    # --- Initialize intrinsics --- 
    fx = fy = 2000
    cx = image_size[0] / 2
    cy = image_size[1] / 2
    K_init = np.array([
        [fx,0,cx],
        [0,fy,cy],
        [0,0,1]
        ], dtype=np.float64)
    dist_init = np.zeros(8)
    flags = (
        cv2.CALIB_USE_INTRINSIC_GUESS
        | cv2.CALIB_FIX_PRINCIPAL_POINT 
        | cv2.CALIB_ZERO_TANGENT_DIST
        )

    # --- Calibrate cameras ---
    rms, K, dist, rvecs, tvecs = cv2.calibrateCamera(
        object_points_list,
        image_points_list,
        image_size,
        K_init,
        dist_init,
        flags=flags
    )

    print("Shared calibration done")
    print("RMS reprojection error:", rms)
    print("Camera matrix K:\n", K)
    print("Distortion coefficients:", dist.ravel())

    # --- Calculate adjusted camera matrix ---
    w,h = image_size
    K_new, _ = cv2.getOptimalNewCameraMatrix(K, dist, (w,h), 1, (w, h))

    # --- Save camera calibration parameters ---
    calib_file = os.path.join(output_folder, file_prefix + 'camera_calibration_params.csv')
    calib_df = pd.DataFrame({
        'image_name': image_files,
        'K': [K]*len(image_files),
        'K_new': [K_new]*len(image_files),
        'distortion_coefficients': [dist]*len(image_files),
        'RMS': [rms]*len(image_files)
    })
    calib_df.to_csv(calib_file, index=False)
    print("Saved calibration params:", calib_file)

    # --- Save undistorted images and camera extrinsics ---
    for i, image_file in enumerate(image_files):
        # Undistort image
        image_undistorted_file = os.path.join(
            output_folder, 
            os.path.splitext(os.path.basename(image_file))[0] + '_undistorted.tiff'
            )
        image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        image_undistorted = cv2.undistort(image, K, dist, None, K_new)
        cv2.imwrite(image_undistorted_file, image_undistorted)

        # Convert rotation matrix from world -> camera to camera -> world
        R_wc = cv2.Rodrigues(rvecs[i])[0]
        R_cw = R_wc.T

        # Calculate camera center
        C = -R_cw @ tvecs[i].reshape(3) + object_points_mean.reshape(3)

        tsai_dict = {
            'fu': K_new[0,0],
            'fv': K_new[1,1],
            'cu': K_new[0,2],
            'cv': K_new[1,2],
            'u_direction': [1,0,0],
            'v_direction': [0,1,0],
            'w_direction': [0,0,1],
            'C': C,
            'R': R_cw,
            'pitch': 1
        }

        tsai_file = os.path.join(
            output_folder, 
            os.path.splitext(os.path.basename(image_file))[0] + '_undistorted.tsai'
            )
        save_tsai(tsai_dict, tsai_file)

    return

os.makedirs(undistorted_folder, exist_ok=True)

# GROUP 1
print('\nGROUP 1: ch01-08\n----------')
image_list = sorted(glob(os.path.join(new_image_folder, '*.tiff')))[0:8]
# calculate shared calibration
print('Optimizing shared camera intrinsics using GCP...')
calibrate_shared_intrinsics(image_list, gcp_merged_ecef_file, undistorted_folder, file_prefix='group1-')

# GROUP 2
print('\nGROUP 2: ch09-16\n----------')
image_list = sorted(glob(os.path.join(new_image_folder, '*.tiff')))[8:]
# calculate shared calibration
print('Optimizing shared camera intrinsics using GCP...')
calibrate_shared_intrinsics(image_list, gcp_merged_ecef_file, undistorted_folder, file_prefix='group2-')

## Initial orthorectification

In [ ]:
os.makedirs(init_ortho_folder, exist_ok=True)

image_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tiff')))
cam_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tsai')))

# Mapproject
for image_file, cam_file in zip(image_list, cam_list):
    image_out_file = os.path.join(init_ortho_folder, os.path.basename(image_file).replace('.tiff', '_map.tiff'))
    cmd = [
        'mapproject',
        '--threads', '12',
        '--nodata-value', '0',
        '--tr', '0.003',
        refdem_file, image_file, cam_file, image_out_file
    ]
    subprocess.run(cmd)

# Mosaic orthoimages
print('\nMosaicking orthoimages')
image_list = sorted(glob(os.path.join(init_ortho_folder, '*.tiff')))
mosaic_file = os.path.join(init_ortho_folder, f'orthomosaic.tif')
fnc = shutil.which('gdal_merge.py')
cmd = [
    'python', fnc,
    '-o', mosaic_file,
    '-n', '0',
    '-a_nodata', '-9999'
] + image_list
subprocess.run(cmd)

## Run stereo preprocessing to create dense match files

In [ ]:
init_stereo_folder = os.path.join(out_folder, 'init_stereo')
os.makedirs(init_stereo_folder, exist_ok=True)

image_list = sorted(glob(os.path.join(undistorted_folder, '*.tiff')))
cam_list = sorted(glob(os.path.join(undistorted_folder, '*.tsai')))

# Set up image pairs
image1_list, image2_list = image_list[0:-1], image_list[1:]
cam1_list, cam2_list = cam_list[0:-1], cam_list[1:]

# skip the 8/9 cams pair (different intrinsics solving during bundle adjust)
iskip = [i for i in range(0,len(image1_list)) if 'ch08' in image1_list[i]][0]
image1_list = image1_list[0:iskip] + image1_list[iskip+1:]
image2_list = image2_list[0:iskip] + image2_list[iskip+1:]
cam1_list = cam1_list[0:iskip] + cam1_list[iskip+1:]
cam2_list = cam2_list[0:iskip] + cam2_list[iskip+1:]

# Iterate over pairs
for i in tqdm(range(len(image1_list))):
    image1, image2 = image1_list[i], image2_list[i]
    cam1, cam2 = cam1_list[i], cam2_list[i]

    pair_prefix = os.path.join(
        init_stereo_folder,
        os.path.splitext(os.path.basename(image1))[0] + '__' + os.path.splitext(os.path.basename(image2))[0],
        'run'
        )
    
    cmd = [
        'parallel_stereo',
        '--threads-singleprocess', '12',
        '--threads-multiprocess', '12',
        '--stop-point', '1',
        image1, image2,
        cam1, cam2,
        pair_prefix,
    ]
    subprocess.run(cmd)

## Bundle adjust

In [ ]:
asp_folder = '/Applications/StereoPipeline-3.6.0-alpha-2025-10-15-arm64-OSX/bin'

ba_folder = os.path.join(out_folder, 'bundle_adjust')
os.makedirs(ba_folder, exist_ok=True)

image_list = sorted(glob(os.path.join(undistorted_folder, '*.tiff')))
cam_list = sorted(glob(os.path.join(undistorted_folder, '*.tsai')))

# Copy dense matches to bundle adjust folder
match_list = sorted(glob(os.path.join(init_stereo_folder, '*', '*.match')))
for match_file in match_list:
    # get image pair
    pair = os.path.dirname(match_file).split('/')[-1]
    # check which group it's in
    first_channel = pair.split('ch')[1][0:2]
    group = 1 if float(first_channel) < 9 else 2
    # define output file
    match_out_file = os.path.join(
        ba_folder, 
        f'run_group{group}-{pair}' + '.match'
        )
    # copy
    _ = shutil.copy2(match_file, match_out_file)

# GROUP 1
# print('\nGROUP 1: ch01-08\n----------')
# image_list_group1 = image_list[0:8]
# cam_list_group1 = cam_list[0:8]
# # Run bundle adjust
# cmd = [
#     os.path.join(asp_folder, 'parallel_bundle_adjust'),
#     '--threads', '12',
#     '--num-iterations', '2000',
#     '--num-passes', '2',
#     '--inline-adjustments',
#     '--force-reuse-match-files',
#     '--fixed-distortion-indices', '2,3',
#     '--heights-from-dem', refdem_file,
#     '--heights-from-dem-uncertainty', '0.01',
#     '--solve-intrinsics',
#     '--intrinsics-to-share', 'optical_center,other_intrinsics',
#     '--intrinsics-to-float', 'all',
#     '-o', os.path.join(ba_folder, 'run_group1')
# ] + image_list_group1 + cam_list_group1
# subprocess.run(cmd)

# GROUP 2
print('\nGROUP 2: ch09-16\n----------')
image_list_group2 = image_list[8:]
cam_list_group2 = cam_list[8:]
# Run bundle adjust
cmd = [
    os.path.join(asp_folder, 'parallel_bundle_adjust'),
    '--threads', '12',
    '--num-iterations', '2000',
    '--num-passes', '2',
    '--inline-adjustments',
    '--force-reuse-match-files',
    '--fixed-distortion-indices', '2,3',
    '--heights-from-dem', refdem_file,
    '--heights-from-dem-uncertainty', '0.01',
    '--solve-intrinsics',
    '--intrinsics-to-share', 'optical_center,other_intrinsics',
    '--intrinsics-to-float', 'all',
    '-o', os.path.join(ba_folder, 'run_group2')
] + image_list_group2 + cam_list_group2
subprocess.run(cmd)

## Final orthorectification

In [ ]:
os.makedirs(final_ortho_folder, exist_ok=True)

image_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tiff')))
cam_list = sorted(glob(os.path.join(ba_folder, '*.tsai')))

# Mapproject
pbar = tqdm(total=len(image_list))
for image_file, cam_file in zip(image_list, cam_list):
    image_out_file = os.path.join(final_ortho_folder, os.path.basename(image_file).replace('.tiff', '_map.tiff'))
    cmd = [
        os.path.join(asp_folder, 'mapproject'),
        '--threads', '12',
        '--nodata-value', '0',
        '--tr', '0.003',
        refdem_file, image_file, cam_file, image_out_file
    ]
    subprocess.run(cmd)
    pbar.update(1)

# Mosaic orthoimages
print('\nMosaicking orthoimages')
image_list = sorted(glob(os.path.join(final_ortho_folder, '*.tiff')))
mosaic_file = os.path.join(final_ortho_folder, f'orthomosaic.tif')
fnc = shutil.which('gdal_merge.py')
cmd = [
    'python', fnc,
    '-o', mosaic_file,
    '-n', '0',
    '-a_nodata', '-9999'
] + image_list
subprocess.run(cmd)


In [ ]:
# Testing hybrid approach

K = np.array([
    [5.70449177e+03, 0.00000000e+00, 1.92000000e+03], 
    [0.00000000e+00, 5.69647956e+03, 1.08000000e+03], 
    [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]
    ])

dist = np.array([-2.32172794, 8.16330791, 0, 0, -15.0270846])

image_file = glob(os.path.join(new_image_folder, '*ch01*.tiff'))[0]
image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)

h,w = image.shape
K_new, roi = cv2.getOptimalNewCameraMatrix(
        K, dist, (w, h), 1, (w, h)
    )
image_undistorted = cv2.undistort(image, K, dist, None, K_new)
image_undistorted_file = os.path.join(out_folder, 'testing', os.path.basename(image_file).replace('.tiff', 'undistorted_full.tiff'))
cv2.imwrite(image_undistorted_file, image_undistorted)

import matplotlib.pyplot as plt
plt.imshow(image_undistorted)

In [ ]:
dist

## Plot current overlap and gaps in data